# Evaluation
Evaluation is a crucial part of building effective AI applications! Without it, we can't understand if our models, prompts, and systems are working as intended. Evaluation is actually a huge topic with many approaches and methodologies - we have separate labs dedicated to exploring this in depth which can be found below: 
* [Evaluation blog](https://community.aws/content/2dHLkZmP7i7m0HjxinUoQPWlyyt/evaluating-llm-systems-best-practices)
* [Hands on evaluation lab](https://github.com/aws-samples/genai-system-evaluation/tree/main)


For this lab, we'll focus on just one exciting technique: LLM-As-A-Judge
This approach uses another LLM with a evaluation rubric to evaluate the results of another model. It lets us automatically assess the quality of model responses based on criteria we define, providing quick feedback without always needing human reviewers.

In the lab, you will:
* Create a grading rubric for your evaluation
* Modify an evaluation dataset
* Run eval on prompts and try to improve them. 


Let's dive in and see how we can use this technique to improve our AI systems!

## Setup

Let's start with our usual imports:

In [ ]:
import boto3
import json
import pandas as pd

# Initialize the Bedrock client
REGION = 'us-west-2'
session = boto3.Session()
bedrock = session.client(service_name='bedrock-runtime', region_name=REGION)

print("✅ Setup complete!")

## Creating an LLM Judge

We're going to evaluate our RAG prompt from the previous workshop to see if it's giving answers we like. We'll run an eval dataset through Bedrock and capture the models response. We'll then run the responses through our evaluation prompt and get metrics we can use to evaluate how well the prompt is performing. 

First, lets start with our LLM-As-A-Judge evaluation prompt

In [ ]:
EVALUATION_SYSTEM_PROMPT = """
You are a fair but discerning evaluator who maintains high standards. 
You use a decimal scoring system (with one decimal place only, e.g., 3.7 or 4.2) to provide nuanced assessments.
Your goal is to differentiate between varying levels of quality using the full range of the scale.
"""

EVALUATION_PROMPT_TEMPLATE = """
Please evaluate the following response using a decimal scoring system.

<question>
{question}
</question>

<context>
{context}
</context>

<generated_response>
{generated_response}
</generated_response>

<gold_standard>
{gold_standard}
</gold_standard>

Evaluation Criteria:
Each criterion is scored on a scale from 0.0 to 1.0 using ONE decimal place only (e.g., 0.7, not 0.75):

1. Context Utilization (0.0-1.0): How well does the response use the information and follow constraints in the context?
   - Compare how effectively the response addresses the key elements in the context
   - Consider how well it follows any specific requirements mentioned

2. Completeness (0.0-1.0): How thoroughly does the response address all aspects of the question?
   - Identify important elements that are covered or missing compared to the gold standard
   - Consider the depth and breadth of coverage

3. Conciseness (0.0-1.0): Is the response appropriately concise without unnecessary information?
   - Evaluate the balance between thoroughness and brevity
   - Consider whether all included information is relevant and necessary

4. Accuracy (0.0-1.0): How factually correct is the information in the response?
   - Check for any factual errors, misconceptions, or imprecisions
   - Consider the technical correctness relative to the gold standard

5. Clarity (0.0-1.0): How clear, well-organized, and easy to understand is the response?
   - Evaluate structure, flow, and accessibility for the intended audience
   - Consider the quality of explanations and examples

Scoring Guidelines:
- Use ONE decimal place only (e.g., 0.7, not 0.75 or 0.67)
- Use this scale as a general guideline:
  * 0.0: Completely fails to meet expectations
  * 0.3: Significantly below expectations
  * 0.5: Partially meets expectations
  * 0.7: Mostly meets expectations
  * 0.9: Exceeds expectations
  * 1.0: Outstanding, on par with gold standard
- Consider scores between these values (0.2, 0.4, 0.6, 0.8) for more nuanced evaluation
- Compare directly with the gold standard to calibrate your scores
- Don't hesitate to use lower scores when warranted, or higher scores when deserved

Evaluation Process:
1. For each criterion, analyze both strengths and weaknesses
2. Be specific in your analysis, referencing the text
3. Assign a decimal score with ONE decimal place for each criterion
4. Calculate the total by adding all five scores (result will be between 0.0-5.0)

Provide your detailed analysis inside <thinking></thinking> tags.
Then give the final score (with one decimal place, e.g., 3.7) inside <score></score> tags.
"""

### Bedrock Helpers
Let's copy paste our helper Bedrock function from the previous workshops

In [ ]:
from typing import Dict, Any, Optional
import re

HAIKU_MODEL_ID: str = "us.anthropic.claude-3-5-haiku-20241022-v1:0"
SONNET_MODEL_ID: str = "us.anthropic.claude-3-5-sonnet-20240620-v1:0"

def call_bedrock(prompt: str, system_prompt: str, model_id: str = HAIKU_MODEL_ID) -> str:
    # Create the message in Bedrock's required format
    user_message: Dict[str, Any] = { "role": "user","content": [{ "text": prompt}] }
    # Configure model parameters
    inference_config: Dict[str, Any] = {
        "temperature": .4,
        "maxTokens": 1000
    }

    # Send request to Claude Haiku 3.5 via Bedrock
    response: Dict[str, Any] = bedrock.converse(
        modelId=model_id,
        messages=[user_message],
        system=[{"text": system_prompt}],
        inferenceConfig=inference_config
    )

    # Get the model's text response
    return response['output']['message']['content'][0]['text']

# Define a function to extract content from XML tags
def extract_tag_content(text: str, tag_name: str) -> Optional[str]:
    """
    Extract content between specified XML tags from text.
    """
    pattern = f'<{tag_name}>(.*?)</{tag_name}>'
    match = re.search(pattern, text, re.DOTALL)
    return match.group(1).strip() if match else None

# How Do We Trust The Judge? 
A common question with LLM-As-A-Judge is "how do we trust it?". To answer that, we need a solution to evaluate the evaluator. This is where rubric alignment comes into play. We'll hand grade a subset of answers based on the same grading rubric the model will use. Using those human curated grades, we can run the same samples through our LLM-As-A-Judge prompt and see how closely they match. 

Lets get started

In [ ]:
# Load evaluation dataset
rubric_alignment_df: pd.DataFrame = pd.read_csv('../data/eval-datasets/rubric_alignment.csv')

rubric_alignment_df

Next lets create our test runner to validate whether or judge prompt aligns to our human evaluators expectations

In [ ]:
import concurrent.futures
from typing import Dict, Tuple, Any

def process_row(row_data: Tuple[int, pd.Series]) -> Dict[str, Any]:
    """Process a single row for evaluation in parallel."""
    index, row = row_data
    prompt = row['query_text']
    context = row['context']
    llm_response = row['llm_response']
    
    # Get the evaluation prompt response
    evaluation_prompt: str = EVALUATION_PROMPT_TEMPLATE.format(
        question=prompt,
        generated_response=llm_response,
        gold_standard=llm_response,
        context=context
    )

    # Get the evaluation response. Note we're using the Sonnet model here.
    evaluation_response: str = call_bedrock(evaluation_prompt, EVALUATION_SYSTEM_PROMPT)

    thinking: str = extract_tag_content(evaluation_response, 'thinking')
    score: str = extract_tag_content(evaluation_response, 'score')
    
    return {
        'index': index,
        'thinking': thinking,
        'score': score
    }

def align_llm_judge_prompt_parallel(df: pd.DataFrame, max_workers: int = 4) -> pd.DataFrame:
    """Parallelize the evaluation process using ThreadPoolExecutor."""
    # make a copy of the dataframe so we don't destroy the existing one between eval runs
    judge_alignment_df = df.copy(deep=True)
    
    # Create a list of row data to process
    row_data = list(judge_alignment_df.iterrows())
    
    # Process rows in parallel
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(executor.map(process_row, row_data))
    
    # Update the dataframe with results
    for result in results:
        index = result['index']
        judge_alignment_df.loc[index, 'ai_reasoning'] = result['thinking']
        judge_alignment_df.loc[index, 'ai_grade'] = result['score']
    
    return judge_alignment_df
        

In [ ]:
judge_alignment_df = align_llm_judge_prompt_parallel(rubric_alignment_df)

judge_alignment_df

Great! We have our answers. However, checking these 1 by 1 would be frustrating at scale. We can do better. Lets create another helper to compare grades and get a more wholistic view of the performance. To do this, we'll use some more standard ML metrics to check how well this aligns.

**Note:** Alignment is hard. What we're looking for is whether the prompt more or less aligns with our expectations and there's no big outliers.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


class RubricValidator:
    def __init__(self, df):
        self.df = df
        self.human_scores = self.convert_to_float(df['human_grade'])
        self.llm_scores = self.convert_to_float(df['ai_grade'])

    def convert_to_float(self, series):
        return series.astype(float)

    def calculate_metrics(self):
        mse = mean_squared_error(self.human_scores, self.llm_scores)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(self.human_scores, self.llm_scores)
        r2 = r2_score(self.human_scores, self.llm_scores)
        
        exact_match = np.mean(self.human_scores == self.llm_scores)
        within_1_point = np.mean(np.abs(self.human_scores - self.llm_scores) <= 1)

        return {
            'MSE': mse,
            'RMSE': rmse,
            'MAE': mae,
            'R-squared': r2,
            'Exact Match Ratio': exact_match,
            'Within 1 Point Ratio': within_1_point
        }

    def generate_report(self):
        metrics = self.calculate_metrics()
        report = "LLM Rubric Validation Report\n"
        report += "===========================\n\n"
        
        for metric, value in metrics.items():
            if isinstance(value, str):
                report += f"{metric}: {value}\n"
            else:
                report += f"{metric}: {value:.4f}\n"
        
        return report

    def analyze_discrepancies(self):
        discrepancies = self.df[self.human_scores != self.llm_scores]
        return discrepancies[['query_text', 'human_grade', 'ai_grade', 'human_reasoning', 'ai_reasoning']]

In [ ]:
# Evaluate the evaluator. 
validator = RubricValidator(judge_alignment_df)
print(validator.generate_report())

# Results
It looks like our LLM judge prompt roughly aligns with our expectations. It's okay if it's not exact. The important part is we believe we can rely on it.

Here are examples of good results:

**Mean Squared Error (MSE) & Root Mean Squared Error (RMSE):**

These metrics measure the average squared difference between the LLM and human grades. They penalize larger errors more heavily, giving us insight into the magnitude of discrepancies.

Interpretation: Lower is better. For a 5-point scale:
* Excellent: MSE < 0.25 (RMSE < 0.5)
* Good: 0.25 ≤ MSE < 1 (0.5 ≤ RMSE < 1)
* Fair: 1 ≤ MSE < 2.25 (1 ≤ RMSE < 1.5)
* Poor: MSE ≥ 2.25 (RMSE ≥ 1.5)

**Within one point**
This is very important. If we see anything below ~90% we need to go back and update our judge prompt. 

### Define our RAG prompt
Next we'll copy paste the prompt from the previous workshop into this workshop. We'll run RAG on the context provided in the gold standard dataset and compare the models results to the gold standard using our rubric. 

In [ ]:
# Define our system prompt. This is the prompt that will be used to generate the response.
RAG_SYSTEM_PROMPT: str = """
You are a coffee expert. You are given a question and a context. 
Your job is to answer the question based ONLY on the context provided. 
Just answer the question, avoid saying "Based on the context provided" before answering.
If the context doesn't contain the answer, say "I don't know"
"""

# Define our RAG prompt template. This is the prompt that will be used to generate the response.
RAG_PROMPT_TEMPLATE: str = """
Using the context below, answer the question.

<context>
{context}
</context>

<question>
{question}
</question>

Remember, if the context doesn't contain the answer, say "I don't know".
"""

# Running an Eval
Now for the fun part. lets run our evaluation dataset and see how it performs! We have a dataset of 6 example questions to start for the workshop. In practice, you'll want at least 50 human curated eval datapoints. But 6 is easier to manage which is why we opted for less.

**Note**: We aren't testing E2E. The dataset assumes the retrieval part of RAG is already done. We simply need to evaluate the prompt. Lets load the eval dataset

In [ ]:
eval_data_df: pd.DataFrame = pd.read_csv('../data/eval-datasets/gold_standard.csv')

eval_data_df

In [ ]:
import concurrent.futures
from typing import Tuple

def process_row(row: pd.Series) -> Tuple[int, str, str]:
    """Process a single row of the dataframe in parallel."""
    index = row.name
    question = row['query_text']
    context = row['context']
    gold_standard = row['gold_standard']
    
    # Get the evaluation prompt response
    rag_prompt: str = RAG_PROMPT_TEMPLATE.format(
        question=question,
        context=context,
    )

    # Get the evaluation response
    rag_response: str = call_bedrock(rag_prompt, RAG_SYSTEM_PROMPT)

    # Now lets evaluate the response against the gold standard.
    evaluation_prompt: str = EVALUATION_PROMPT_TEMPLATE.format(
        question=question,
        generated_response=rag_response,
        gold_standard=gold_standard,
        context=context
    )

    # Get the evaluation response. Note we're using the Sonnet model here.
    evaluation_response: str = call_bedrock(evaluation_prompt, EVALUATION_SYSTEM_PROMPT)

    thinking: str = extract_tag_content(evaluation_response, 'thinking')
    score: str = extract_tag_content(evaluation_response, 'score')
    
    return index, thinking, score

def run_eval(df: pd.DataFrame, max_workers: int = 4) -> pd.DataFrame:
    """
    Run evaluation on all rows in parallel.
    """
    # make a copy of the dataframe so we don't destroy the existing one between eval runs
    judge_alignment_df = df.copy(deep=True)
    
    # Use ThreadPoolExecutor to process rows in parallel
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all jobs to the executor
        future_to_row = {executor.submit(process_row, row): row for _, row in df.iterrows()}
        
        # Process results as they complete
        for future in concurrent.futures.as_completed(future_to_row):
            try:
                index, thinking, score = future.result()
                # Add the evaluation response to the dataframe
                judge_alignment_df.loc[index, 'ai_reasoning'] = thinking
                judge_alignment_df.loc[index, 'ai_grade'] = score
                
                # Print progress (optional)
                print(f"Completed evaluation {index+1}/{len(df)}")
                
            except Exception as e:
                print(f"Error processing row: {e}")
    
    return judge_alignment_df

In [ ]:
# Run the eval
eval_results_df: pd.DataFrame = run_eval(eval_data_df)

eval_results_df

Similar to our rubric alignment, lets create a helper class to generate a report of how the prompt is behaving. 

In [ ]:
import pandas as pd
import numpy as np
from textwrap import fill

class PromptEvaluator:
    def __init__(self, df):
        self.df = df
        self.grades = df['ai_grade'].astype(float)
    
    def calculate_metrics(self):
        return {
            'Mean': np.mean(self.grades),
            'Median': np.median(self.grades),
            'Standard Deviation': np.std(self.grades),
            'Minimum Grade': np.min(self.grades),
            'Maximum Grade': np.max(self.grades)
        }
    
    def generate_report(self):
        metrics = self.calculate_metrics()
        report = "Prompt Evaluation Report\n"
        report += "========================\n\n"
        
        for metric, value in metrics.items():
            report += f"{metric}: {value:.2f}\n"
        
        return report
    
    def analyze_grade_distribution(self):
        # Create bins for grades (1-2, 2-3, 3-4, 4-5)
        bins = [0, 1, 2, 3, 4, 5]
        labels = ['0-1', '1-2', '2-3', '3-4', '4-5']
        
        # Create a new column with binned grades
        binned_grades = pd.cut(self.df['ai_grade'].astype(float), 
                            bins=bins, 
                            labels=labels, 
                            include_lowest=True,
                            right=True)
        
        # Count occurrences in each bin
        distribution = binned_grades.value_counts().sort_index()
        
        return distribution

    def pretty_print_lowest_results(self, n=3, width=80):
        lowest_results = self.df.nsmallest(n, 'grade')
        for index, row in lowest_results.iterrows():
            print(f"{'='*width}\n")
            print(f"Grade: {row['ai_grade']}\n")
            print("Query Text:")
            print(fill(row['query_text'], width=width))
            print("\nLLM Response:")
            print(fill(row['llm_response'], width=width))
            print("\nReasoning:")
            print(fill(row['reasoning'], width=width))
            print(f"\n{'='*width}\n")

In [ ]:
# Run the evaluator
evaluator = PromptEvaluator(eval_results_df)

# Generate and print the report
print(evaluator.generate_report())

# Analyze grade distribution
print(evaluator.analyze_grade_distribution())

## Exercise

The results don't look great. That's to be expected though, this is not a very good RAG prompt 😊. Most of the scores are falling below what we would consider acceptable quality.

Now it's your turn! Your goal is to improve the RAG prompt to get better evaluation scores. Here's what to do:

1. Analyze the current RAG prompt to identify its weaknesses.

2. Create an improved version of the RAG prompt that addresses these weaknesses. Consider:
   - Adding more specific instructions
   - Providing better context utilization guidance
   - Incorporating structure for more complete answers
   - Specifying the desired tone and style

3. Test your improved prompt with the same set of questions.

4. Use the evaluation system to measure your improvements.

Your target is to get all responses scoring between 3-5 on our evaluation scale, with the majority falling in the 4-5 range.

Remember: Good prompt engineering is iterative. Analyze the results, identify patterns in lower-scoring responses, and continue refining your prompt until you consistently achieve high-quality outputs!

# Conclusion
This concludes module 1. Next lets get into some more advanced concepts and introduce our first framework (LangGraph)